In [ ]:
#Integrantes del grupo

# Flavia Morena Valentina Pacheco Jaime
# Jozek Andrzej Hajduk Sánchez
# Jorge Andres Rangel Zapata

In [1]:
import psycopg2
import pandas as pd

from google.cloud import bigquery
from google.oauth2 import service_account

In [6]:
conn = psycopg2.connect( host = 'localhost', port = 5432, database = 'Adventure', user = 'jorgedevops' )

In [13]:
cur = conn.cursor()
Address = pd.read_sql_query( 'SELECT st.name , count(pp.businessentityid) FROM sales.salesterritory st, sales.salesorderheader so, person.person pp WHERE pp.businessentityid = so.customerid AND st.territoryid  = so.territoryid group by (st.name)', conn )
cur.close()



In [14]:
Address

,name,count
0,France,1552
1,Southwest,3140
2,Northwest,2379
3,United Kingdom,2041
4,Northeast,7
5,Central,5
6,Australia,4283
7,Germany,1567
8,Southeast,13
9,Canada,2476


In [9]:

cur = conn.cursor()
Countryregion = pd.read_sql_query( 'SELECT * FROM person.countryregion', conn )
cur.close()

In [10]:

cur = conn.cursor()
Stateprovince = pd.read_sql_query( 'SELECT * FROM person.stateprovince', conn )
cur.close()

In [11]:

cur = conn.cursor()
Addresstype = pd.read_sql_query( 'SELECT * FROM person.addresstype', conn )
cur.close()


In [12]:
coon.close()

Series([], Name: addressid, dtype: object)

[(1, 'COL', '2020-05-02 15:00:00'),
 (2, 'USA', '2020-05-02 20:00:00'),
 (3, 'MEX', '2020-05-02 10:00:00'),
 (4, 'ARG', '2020-05-02 11:00:00'),
 (5, 'CHI', '2020-05-02 12:00:00')]

,ID,country_code,Date
0,1,COL,2020-05-02 15:00:00
1,2,USA,2020-05-02 20:00:00
2,3,MEX,2020-05-02 10:00:00
3,4,ARG,2020-05-02 11:00:00
4,5,CHI,2020-05-02 12:00:00


NameError: name 's' is not defined

NameError: name 's' is not defined

NameError: name 's' is not defined

NameError: name 's' is not defined